In [1]:
#dependencies
import re
from bs4 import BeautifulSoup 
from splinter import Browser
import requests
import pandas as pd
from selenium import webdriver 
from selenium.common.exceptions import TimeoutException
from webdriver_manager.chrome import ChromeDriverManager
from selenium.webdriver.chrome.service import Service as BraveService
from webdriver_manager.core.utils import ChromeType
import json
import time
from tqdm.notebook import tqdm 

<h4>Start cells here if {state}_geo file has not been FIPS encoded

In [2]:
fips_check = pd.read_csv("../../data/state_data/geo/geocoded/geo_MP.csv") #- original geocoded file
fips_check = fips_check.drop(["Unnamed: 0.1", "Unnamed: 0.1.1"], axis =1) 
fips_check

,Unnamed: 0,LoanNumber,DateApproved,SBAOfficeCode,ProcessingMethod,BorrowerName,BorrowerAddress,BorrowerCity,BorrowerState,BorrowerZip,...,TRIIndustrySector,IndustrySubsector,4DigitNAICS,SBAOfficeLabel,Loan_Range,job_Range,full_add,Lat,Long,FIPS_z
0,514903,5731797102,4/14/20,951,PPP,"ASIA PACIFIC HOTELS, INC",PO Box 501029,SAIPAN,MP,96950.0,...,999 Other,7211 Traveler Accommodation,7211.0,HAWAII DISTRICT OFFICE,d $2-5 million,100 to 249,"PO Box 501029, SAIPAN, MP",15.185048,145.746726,NaN
1,514904,4500217210,4/27/20,951,PPP,INTERPACIFIC RESORTS (SAIPAN) CORPORATION,502370 PO Box,SAIPAN,MP,96950.0,...,999 Other,7211 Traveler Accommodation,7211.0,HAWAII DISTRICT OFFICE,c $1-2 million,250 to 499,"502370 PO Box, SAIPAN, MP",15.185048,145.746726,NaN
2,514905,5843458509,3/2/21,951,PPS,MICRONESIA RESORT INC.,CHALAN PALE ARNOLD SAN ROQUE,SAIPAN,MP,96950.0,...,999 Other,7211 Traveler Accommodation,7211.0,HAWAII DISTRICT OFFICE,c $1-2 million,100 to 249,"CHALAN PALE ARNOLD SAN ROQUE, SAIPAN, MP",15.252495,145.779802,NaN
3,514906,4511357201,4/27/20,951,PPP,MICRONESIA RESORT INC.,5152 PO BOX,SAIPAN,MP,96950.0,...,999 Other,7211 Traveler Accommodation,7211.0,HAWAII DISTRICT OFFICE,c $1-2 million,250 to 499,"5152 PO BOX, SAIPAN, MP",15.185048,145.746726,NaN
4,514907,8497628610,3/25/21,951,PPS,WORLD CORPORATION,World Resort Building Susupe Village Beach Road,Saipan,MP,96950.0,...,999 Other,7211 Traveler Accommodation,7211.0,HAWAII DISTRICT OFFICE,c $1-2 million,250 to 499,World Resort Building Susupe Village Beach Roa...,15.158430,145.703139,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
77,514980,5679237307,4/30/20,951,PPP,"PACIFIC AMUSEMENT, INC.",Chalan Pale Arnold Gualo Rai,Saipan,MP,96950.0,...,999 Other,7112 Spectator Sports,7112.0,HAWAII DISTRICT OFFICE,"a $150,000-350,000",20 to 49,"Chalan Pale Arnold Gualo Rai, Saipan, MP",15.194064,145.719844,NaN
78,514981,4304997202,4/27/20,951,PPP,KALAYAAN INC.,PO Box 505656 CK CK,SAIPAN,MP,96950.0,...,311 Food,3119 Other Food Manufacturing,3119.0,HAWAII DISTRICT OFFICE,"a $150,000-350,000",20 to 49,"PO Box 505656 CK CK, SAIPAN, MP",15.185048,145.746726,NaN
79,514982,6670117402,5/15/20,951,PPP,"TANO GROUP, INC.",PO Box 5017 CHRB,SAIPAN,MP,96950.0,...,999 Other,2361 Residential Building Construction,2361.0,HAWAII DISTRICT OFFICE,"a $150,000-350,000",20 to 49,"PO Box 5017 CHRB, SAIPAN, MP",15.185048,145.746726,NaN
80,514983,9968828210,1/15/21,951,PPS,"YOUNIS ART STUDIO, INC.",Alaihai Avenue,Saipan,MP,96950.0,...,511 512 519 Publishing,"5111 Newspaper, Periodical, Book, and Director...",5111.0,HAWAII DISTRICT OFFICE,"a $150,000-350,000",20 to 49,"Alaihai Avenue, Saipan, MP",15.199415,145.718000,NaN


In [3]:
columns = ['LoanNumber','full_add', 'Lat', 'Long']
fips_check = fips_check[columns]

In [4]:
fips_check_nan = fips_check[fips_check["Lat"].isnull()] # check to make sure no nulls in coords
fips_check_nan #3

,LoanNumber,full_add,Lat,Long


In [5]:
fips_check_notna = fips_check[fips_check["Lat"].notna()] # check to make sure no nulls in coords
fips_check_notna.head() #15866

,LoanNumber,full_add,Lat,Long
0,5731797102,"PO Box 501029, SAIPAN, MP",15.185048,145.746726
1,4500217210,"502370 PO Box, SAIPAN, MP",15.185048,145.746726
2,5843458509,"CHALAN PALE ARNOLD SAN ROQUE, SAIPAN, MP",15.252495,145.779802
3,4511357201,"5152 PO BOX, SAIPAN, MP",15.185048,145.746726
4,8497628610,World Resort Building Susupe Village Beach Roa...,15.158430,145.703139


<h4> Start cells here if partial FIPS already available in {state}_fips_scraped file

In [ ]:
fips_check_scraped = pd.read_csv("../../data/state_data/geo/geo_fips/MP_fips_scraped.csv") 
fips_check_scraped.head() #15869

In [ ]:
columns = ['LoanNumber','full_add', 'Lat', 'Long', 'FIPS_z']
fips_check_scraped = fips_check_scraped[columns]

In [28]:
#pd.set_option('display.float_format', '{:.0f}'.format)

In [ ]:
#Latitude, Longitude, not included
fips_check_scraped_notna = fips_check_scraped[fips_check_scraped["Lat"].notna()] # check to make sure no nulls in coords
fips_check_scraped_notna #15866

In [ ]:
#FIPS not included, of Lat, Long available
fips_check_scraped_null = fips_check_scraped_notna[fips_check_scraped_notna["FIPS_z"].isnull()] # check to make sure no nulls in coords
fips_check_scraped_null #5503

<h4>FIPS Script

In [9]:
#if geocode file
lat = fips_check_notna["Lat"].astype(float).astype(str).to_list()
long = fips_check_notna["Long"].astype(float).astype(str).to_list()

#if FIPS_z file 
#lat = fips_check_scraped_null["Lat"].astype(float).astype(str).to_list()
#long = fips_check_scraped_null["Long"].astype(float).astype(str).to_list()

In [1]:
#before running script, check for any NaN within Lat, Long fields. 
def sleep():
    time.sleep(1)
option = webdriver.ChromeOptions()
option.binary_location = "/Applications/Brave Browser.app/Contents/MacOS/Brave Browser"
driver = Browser('chrome', options=option, headless=False)    
url = 'https://geocoding.geo.census.gov/geocoder/geographies/coordinates?form'
driver.visit(url)

not_found = []
geo_id = []

i=0
for i in tqdm(range(len(lat))):
    #while i < len(lat):
        driver.find_by_id('geoCoordX').clear()
        driver.find_by_id('geoCoordY').clear()
        driver.find_by_id('geoCoordX').fill(long[i])
        driver.find_by_id('geoCoordY').fill(lat[i])
        driver.find_by_id('geoCoordsSubmit').click()
        #sleep()
        #html = driver.html
        soup = BeautifulSoup(driver.html, 'html.parser')


        results = soup.find_all('div', id='geoCoordsJsonOutput')
        for result in results:
            result_text = result.text

            result_digits = re.findall('\d+', result_text)
            startswith = [s for s in result_digits if s.startswith('69')] #VI
            fips_elements =[element for element in startswith if len(element)>=15]
            
            try:
                #print(f"{i},{lat[i]},{long[i]} is {fips_elements}")
                geo_id.append(fips_elements)
            except (KeyError, IndexError):
                #print(f"{i},{lat[i]},{long[i]} Not Found")
                not_found.append(i)
                geo_id.append(None)  
            except (TimeoutException):
                driver.navigate.refresh()
    
        #sleep()
        i=i+1

NameError: name 'webdriver' is not defined

coordinates wtih more than one return:
385,37.514070000000004,-77.35472 is ['514070000000004', '510872015041003'] 
1178,37.516454700000004,-77.8100575 is ['516454700000004', '511455001022014'] 
2660,37.519462299999994,-77.340384 is ['519462299999994', '510872014061012']
second option reference FIPS code, first option references LatCoord, update fields in DataFrame

https://iqss.github.io/dss-webscrape/filling-in-web-forms.html

In [11]:
geo_id_list =geo_id 

Length Check

In [12]:
print(len(geo_id_list))

82


Matching FIPS to DataFrame

In [13]:
#for geocode file
fips_fill = fips_check_notna #confirm pair to notna DataFrame
#if FIPS available
#fips_fill = fips_check_scraped_null.iloc[:25062]

In [14]:
fips_fill["FIPS_z"] = geo_id_list
fips_fill

,LoanNumber,full_add,Lat,Long,FIPS_z
0,5731797102,"PO Box 501029, SAIPAN, MP",15.185048,145.746726,[691100015001000]
1,4500217210,"502370 PO Box, SAIPAN, MP",15.185048,145.746726,[691100015001000]
2,5843458509,"CHALAN PALE ARNOLD SAN ROQUE, SAIPAN, MP",15.252495,145.779802,[691100001002005]
3,4511357201,"5152 PO BOX, SAIPAN, MP",15.185048,145.746726,[691100015001000]
4,8497628610,World Resort Building Susupe Village Beach Roa...,15.158430,145.703139,[691100008002003]
...,...,...,...,...,...
77,5679237307,"Chalan Pale Arnold Gualo Rai, Saipan, MP",15.194064,145.719844,[691100004002000]
78,4304997202,"PO Box 505656 CK CK, SAIPAN, MP",15.185048,145.746726,[691100015001000]
79,6670117402,"PO Box 5017 CHRB, SAIPAN, MP",15.185048,145.746726,[691100015001000]
80,9968828210,"Alaihai Avenue, Saipan, MP",15.199415,145.718000,[691100004002009]


Remove brackets

In [15]:
fips_fill['FIPS_z'] = fips_fill['FIPS_z'].str[0]
fips_fill

,LoanNumber,full_add,Lat,Long,FIPS_z
0,5731797102,"PO Box 501029, SAIPAN, MP",15.185048,145.746726,691100015001000
1,4500217210,"502370 PO Box, SAIPAN, MP",15.185048,145.746726,691100015001000
2,5843458509,"CHALAN PALE ARNOLD SAN ROQUE, SAIPAN, MP",15.252495,145.779802,691100001002005
3,4511357201,"5152 PO BOX, SAIPAN, MP",15.185048,145.746726,691100015001000
4,8497628610,World Resort Building Susupe Village Beach Roa...,15.158430,145.703139,691100008002003
...,...,...,...,...,...
77,5679237307,"Chalan Pale Arnold Gualo Rai, Saipan, MP",15.194064,145.719844,691100004002000
78,4304997202,"PO Box 505656 CK CK, SAIPAN, MP",15.185048,145.746726,691100015001000
79,6670117402,"PO Box 5017 CHRB, SAIPAN, MP",15.185048,145.746726,691100015001000
80,9968828210,"Alaihai Avenue, Saipan, MP",15.199415,145.718000,691100004002009


In [16]:
fips_fill.loc[fips_fill["FIPS_z"].isnull()] #check #24

,LoanNumber,full_add,Lat,Long,FIPS_z


Parse for Merge

In [17]:
cols= ["LoanNumber", "FIPS_z"]
fips_fill_parsed = fips_fill[cols]
fips_fill_parsed

,LoanNumber,FIPS_z
0,5731797102,691100015001000
1,4500217210,691100015001000
2,5843458509,691100001002005
3,4511357201,691100015001000
4,8497628610,691100008002003
...,...,...
77,5679237307,691100004002000
78,4304997202,691100015001000
79,6670117402,691100015001000
80,9968828210,691100004002009


Read in base data

In [18]:
#if from geocode file
fips_check = pd.read_csv("../../data/state_data/geo/geocoded/geo_MP.csv")
#fips_check = fips_check.drop(["Unnamed: 0.1", "Unnamed: 0.1.1"], axis =1) 
fips_check.head()

#if from FIPS file
#fips_check = pd.read_csv("../../data/state_data/geo/geo_fips/MP_fips_scraped.csv") 
#fips_check = fips_check.drop(["Unnamed: 0.1", "Unnamed: 0.1.1", "Unnamed: 0"], axis =1) 
#fips_check.head() #22826


,Unnamed: 0,Unnamed: 0.1,Unnamed: 0.1.1,LoanNumber,DateApproved,SBAOfficeCode,ProcessingMethod,BorrowerName,BorrowerAddress,BorrowerCity,...,TRIIndustrySector,IndustrySubsector,4DigitNAICS,SBAOfficeLabel,Loan_Range,job_Range,full_add,Lat,Long,FIPS_z
0,514903,514903,514903,5731797102,4/14/20,951,PPP,"ASIA PACIFIC HOTELS, INC",PO Box 501029,SAIPAN,...,999 Other,7211 Traveler Accommodation,7211.0,HAWAII DISTRICT OFFICE,d $2-5 million,100 to 249,"PO Box 501029, SAIPAN, MP",15.185048,145.746726,NaN
1,514904,514904,514904,4500217210,4/27/20,951,PPP,INTERPACIFIC RESORTS (SAIPAN) CORPORATION,502370 PO Box,SAIPAN,...,999 Other,7211 Traveler Accommodation,7211.0,HAWAII DISTRICT OFFICE,c $1-2 million,250 to 499,"502370 PO Box, SAIPAN, MP",15.185048,145.746726,NaN
2,514905,514905,514905,5843458509,3/2/21,951,PPS,MICRONESIA RESORT INC.,CHALAN PALE ARNOLD SAN ROQUE,SAIPAN,...,999 Other,7211 Traveler Accommodation,7211.0,HAWAII DISTRICT OFFICE,c $1-2 million,100 to 249,"CHALAN PALE ARNOLD SAN ROQUE, SAIPAN, MP",15.252495,145.779802,NaN
3,514906,514906,514906,4511357201,4/27/20,951,PPP,MICRONESIA RESORT INC.,5152 PO BOX,SAIPAN,...,999 Other,7211 Traveler Accommodation,7211.0,HAWAII DISTRICT OFFICE,c $1-2 million,250 to 499,"5152 PO BOX, SAIPAN, MP",15.185048,145.746726,NaN
4,514907,514907,514907,8497628610,3/25/21,951,PPS,WORLD CORPORATION,World Resort Building Susupe Village Beach Road,Saipan,...,999 Other,7211 Traveler Accommodation,7211.0,HAWAII DISTRICT OFFICE,c $1-2 million,250 to 499,World Resort Building Susupe Village Beach Roa...,15.158430,145.703139,NaN


Merge with MainFrame

In [19]:
fips_merge = fips_check.merge(fips_fill_parsed, on="LoanNumber", how="left")
fips_merge

,Unnamed: 0,Unnamed: 0.1,Unnamed: 0.1.1,LoanNumber,DateApproved,SBAOfficeCode,ProcessingMethod,BorrowerName,BorrowerAddress,BorrowerCity,...,IndustrySubsector,4DigitNAICS,SBAOfficeLabel,Loan_Range,job_Range,full_add,Lat,Long,FIPS_z_x,FIPS_z_y
0,514903,514903,514903,5731797102,4/14/20,951,PPP,"ASIA PACIFIC HOTELS, INC",PO Box 501029,SAIPAN,...,7211 Traveler Accommodation,7211.0,HAWAII DISTRICT OFFICE,d $2-5 million,100 to 249,"PO Box 501029, SAIPAN, MP",15.185048,145.746726,NaN,691100015001000
1,514904,514904,514904,4500217210,4/27/20,951,PPP,INTERPACIFIC RESORTS (SAIPAN) CORPORATION,502370 PO Box,SAIPAN,...,7211 Traveler Accommodation,7211.0,HAWAII DISTRICT OFFICE,c $1-2 million,250 to 499,"502370 PO Box, SAIPAN, MP",15.185048,145.746726,NaN,691100015001000
2,514905,514905,514905,5843458509,3/2/21,951,PPS,MICRONESIA RESORT INC.,CHALAN PALE ARNOLD SAN ROQUE,SAIPAN,...,7211 Traveler Accommodation,7211.0,HAWAII DISTRICT OFFICE,c $1-2 million,100 to 249,"CHALAN PALE ARNOLD SAN ROQUE, SAIPAN, MP",15.252495,145.779802,NaN,691100001002005
3,514906,514906,514906,4511357201,4/27/20,951,PPP,MICRONESIA RESORT INC.,5152 PO BOX,SAIPAN,...,7211 Traveler Accommodation,7211.0,HAWAII DISTRICT OFFICE,c $1-2 million,250 to 499,"5152 PO BOX, SAIPAN, MP",15.185048,145.746726,NaN,691100015001000
4,514907,514907,514907,8497628610,3/25/21,951,PPS,WORLD CORPORATION,World Resort Building Susupe Village Beach Road,Saipan,...,7211 Traveler Accommodation,7211.0,HAWAII DISTRICT OFFICE,c $1-2 million,250 to 499,World Resort Building Susupe Village Beach Roa...,15.158430,145.703139,NaN,691100008002003
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
77,514980,514980,514980,5679237307,4/30/20,951,PPP,"PACIFIC AMUSEMENT, INC.",Chalan Pale Arnold Gualo Rai,Saipan,...,7112 Spectator Sports,7112.0,HAWAII DISTRICT OFFICE,"a $150,000-350,000",20 to 49,"Chalan Pale Arnold Gualo Rai, Saipan, MP",15.194064,145.719844,NaN,691100004002000
78,514981,514981,514981,4304997202,4/27/20,951,PPP,KALAYAAN INC.,PO Box 505656 CK CK,SAIPAN,...,3119 Other Food Manufacturing,3119.0,HAWAII DISTRICT OFFICE,"a $150,000-350,000",20 to 49,"PO Box 505656 CK CK, SAIPAN, MP",15.185048,145.746726,NaN,691100015001000
79,514982,514982,514982,6670117402,5/15/20,951,PPP,"TANO GROUP, INC.",PO Box 5017 CHRB,SAIPAN,...,2361 Residential Building Construction,2361.0,HAWAII DISTRICT OFFICE,"a $150,000-350,000",20 to 49,"PO Box 5017 CHRB, SAIPAN, MP",15.185048,145.746726,NaN,691100015001000
80,514983,514983,514983,9968828210,1/15/21,951,PPS,"YOUNIS ART STUDIO, INC.",Alaihai Avenue,Saipan,...,"5111 Newspaper, Periodical, Book, and Director...",5111.0,HAWAII DISTRICT OFFICE,"a $150,000-350,000",20 to 49,"Alaihai Avenue, Saipan, MP",15.199415,145.718000,NaN,691100004002009


<b>Fill FIPS Columns - if FIPS file

In [20]:
fips_merge["FIPS_z"] = fips_merge["FIPS_z_y"].fillna(fips_merge["FIPS_z_x"])
fips_merge

,Unnamed: 0,Unnamed: 0.1,Unnamed: 0.1.1,LoanNumber,DateApproved,SBAOfficeCode,ProcessingMethod,BorrowerName,BorrowerAddress,BorrowerCity,...,4DigitNAICS,SBAOfficeLabel,Loan_Range,job_Range,full_add,Lat,Long,FIPS_z_x,FIPS_z_y,FIPS_z
0,514903,514903,514903,5731797102,4/14/20,951,PPP,"ASIA PACIFIC HOTELS, INC",PO Box 501029,SAIPAN,...,7211.0,HAWAII DISTRICT OFFICE,d $2-5 million,100 to 249,"PO Box 501029, SAIPAN, MP",15.185048,145.746726,NaN,691100015001000,691100015001000
1,514904,514904,514904,4500217210,4/27/20,951,PPP,INTERPACIFIC RESORTS (SAIPAN) CORPORATION,502370 PO Box,SAIPAN,...,7211.0,HAWAII DISTRICT OFFICE,c $1-2 million,250 to 499,"502370 PO Box, SAIPAN, MP",15.185048,145.746726,NaN,691100015001000,691100015001000
2,514905,514905,514905,5843458509,3/2/21,951,PPS,MICRONESIA RESORT INC.,CHALAN PALE ARNOLD SAN ROQUE,SAIPAN,...,7211.0,HAWAII DISTRICT OFFICE,c $1-2 million,100 to 249,"CHALAN PALE ARNOLD SAN ROQUE, SAIPAN, MP",15.252495,145.779802,NaN,691100001002005,691100001002005
3,514906,514906,514906,4511357201,4/27/20,951,PPP,MICRONESIA RESORT INC.,5152 PO BOX,SAIPAN,...,7211.0,HAWAII DISTRICT OFFICE,c $1-2 million,250 to 499,"5152 PO BOX, SAIPAN, MP",15.185048,145.746726,NaN,691100015001000,691100015001000
4,514907,514907,514907,8497628610,3/25/21,951,PPS,WORLD CORPORATION,World Resort Building Susupe Village Beach Road,Saipan,...,7211.0,HAWAII DISTRICT OFFICE,c $1-2 million,250 to 499,World Resort Building Susupe Village Beach Roa...,15.158430,145.703139,NaN,691100008002003,691100008002003
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
77,514980,514980,514980,5679237307,4/30/20,951,PPP,"PACIFIC AMUSEMENT, INC.",Chalan Pale Arnold Gualo Rai,Saipan,...,7112.0,HAWAII DISTRICT OFFICE,"a $150,000-350,000",20 to 49,"Chalan Pale Arnold Gualo Rai, Saipan, MP",15.194064,145.719844,NaN,691100004002000,691100004002000
78,514981,514981,514981,4304997202,4/27/20,951,PPP,KALAYAAN INC.,PO Box 505656 CK CK,SAIPAN,...,3119.0,HAWAII DISTRICT OFFICE,"a $150,000-350,000",20 to 49,"PO Box 505656 CK CK, SAIPAN, MP",15.185048,145.746726,NaN,691100015001000,691100015001000
79,514982,514982,514982,6670117402,5/15/20,951,PPP,"TANO GROUP, INC.",PO Box 5017 CHRB,SAIPAN,...,2361.0,HAWAII DISTRICT OFFICE,"a $150,000-350,000",20 to 49,"PO Box 5017 CHRB, SAIPAN, MP",15.185048,145.746726,NaN,691100015001000,691100015001000
80,514983,514983,514983,9968828210,1/15/21,951,PPS,"YOUNIS ART STUDIO, INC.",Alaihai Avenue,Saipan,...,5111.0,HAWAII DISTRICT OFFICE,"a $150,000-350,000",20 to 49,"Alaihai Avenue, Saipan, MP",15.199415,145.718000,NaN,691100004002009,691100004002009


Drop Extra FIPS Columns

In [21]:
fips_merge_drop = fips_merge.drop(["FIPS_z_x", "FIPS_z_y"], axis=1)
#fips_merge_drop = fips_merge_drop.drop(["Unnamed: 0.1", "Unnamed: 0.1.1.1"], axis=1)
fips_merge_drop

,Unnamed: 0,Unnamed: 0.1,Unnamed: 0.1.1,LoanNumber,DateApproved,SBAOfficeCode,ProcessingMethod,BorrowerName,BorrowerAddress,BorrowerCity,...,TRIIndustrySector,IndustrySubsector,4DigitNAICS,SBAOfficeLabel,Loan_Range,job_Range,full_add,Lat,Long,FIPS_z
0,514903,514903,514903,5731797102,4/14/20,951,PPP,"ASIA PACIFIC HOTELS, INC",PO Box 501029,SAIPAN,...,999 Other,7211 Traveler Accommodation,7211.0,HAWAII DISTRICT OFFICE,d $2-5 million,100 to 249,"PO Box 501029, SAIPAN, MP",15.185048,145.746726,691100015001000
1,514904,514904,514904,4500217210,4/27/20,951,PPP,INTERPACIFIC RESORTS (SAIPAN) CORPORATION,502370 PO Box,SAIPAN,...,999 Other,7211 Traveler Accommodation,7211.0,HAWAII DISTRICT OFFICE,c $1-2 million,250 to 499,"502370 PO Box, SAIPAN, MP",15.185048,145.746726,691100015001000
2,514905,514905,514905,5843458509,3/2/21,951,PPS,MICRONESIA RESORT INC.,CHALAN PALE ARNOLD SAN ROQUE,SAIPAN,...,999 Other,7211 Traveler Accommodation,7211.0,HAWAII DISTRICT OFFICE,c $1-2 million,100 to 249,"CHALAN PALE ARNOLD SAN ROQUE, SAIPAN, MP",15.252495,145.779802,691100001002005
3,514906,514906,514906,4511357201,4/27/20,951,PPP,MICRONESIA RESORT INC.,5152 PO BOX,SAIPAN,...,999 Other,7211 Traveler Accommodation,7211.0,HAWAII DISTRICT OFFICE,c $1-2 million,250 to 499,"5152 PO BOX, SAIPAN, MP",15.185048,145.746726,691100015001000
4,514907,514907,514907,8497628610,3/25/21,951,PPS,WORLD CORPORATION,World Resort Building Susupe Village Beach Road,Saipan,...,999 Other,7211 Traveler Accommodation,7211.0,HAWAII DISTRICT OFFICE,c $1-2 million,250 to 499,World Resort Building Susupe Village Beach Roa...,15.158430,145.703139,691100008002003
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
77,514980,514980,514980,5679237307,4/30/20,951,PPP,"PACIFIC AMUSEMENT, INC.",Chalan Pale Arnold Gualo Rai,Saipan,...,999 Other,7112 Spectator Sports,7112.0,HAWAII DISTRICT OFFICE,"a $150,000-350,000",20 to 49,"Chalan Pale Arnold Gualo Rai, Saipan, MP",15.194064,145.719844,691100004002000
78,514981,514981,514981,4304997202,4/27/20,951,PPP,KALAYAAN INC.,PO Box 505656 CK CK,SAIPAN,...,311 Food,3119 Other Food Manufacturing,3119.0,HAWAII DISTRICT OFFICE,"a $150,000-350,000",20 to 49,"PO Box 505656 CK CK, SAIPAN, MP",15.185048,145.746726,691100015001000
79,514982,514982,514982,6670117402,5/15/20,951,PPP,"TANO GROUP, INC.",PO Box 5017 CHRB,SAIPAN,...,999 Other,2361 Residential Building Construction,2361.0,HAWAII DISTRICT OFFICE,"a $150,000-350,000",20 to 49,"PO Box 5017 CHRB, SAIPAN, MP",15.185048,145.746726,691100015001000
80,514983,514983,514983,9968828210,1/15/21,951,PPS,"YOUNIS ART STUDIO, INC.",Alaihai Avenue,Saipan,...,511 512 519 Publishing,"5111 Newspaper, Periodical, Book, and Director...",5111.0,HAWAII DISTRICT OFFICE,"a $150,000-350,000",20 to 49,"Alaihai Avenue, Saipan, MP",15.199415,145.718000,691100004002009


In [22]:
#fips_merge_drop = fips_merge.drop(["Unnamed: 0.1", "Unnamed: 0.1.1.1"], axis=1)
fips_merge_drop.loc[fips_merge["FIPS_z"].isnull()] #verify #27 , notna #22799 / 22826

,Unnamed: 0,Unnamed: 0.1,Unnamed: 0.1.1,LoanNumber,DateApproved,SBAOfficeCode,ProcessingMethod,BorrowerName,BorrowerAddress,BorrowerCity,...,TRIIndustrySector,IndustrySubsector,4DigitNAICS,SBAOfficeLabel,Loan_Range,job_Range,full_add,Lat,Long,FIPS_z


Convert to File

In [23]:
fips_MP = fips_merge_drop

In [24]:
fips_MP.to_csv("../../data/state_data/geo/geo_fips/MP_fips_scraped.csv") 

Review & Compare

In [25]:
geo_MP_fips = pd.read_csv("../../data/state_data/geo/geo_fips/MP_fips_scraped.csv") 
geo_MP_fips.head()

,Unnamed: 0,Unnamed: 0.1,Unnamed: 0.1.1,Unnamed: 0.1.1.1,LoanNumber,DateApproved,SBAOfficeCode,ProcessingMethod,BorrowerName,BorrowerAddress,...,TRIIndustrySector,IndustrySubsector,4DigitNAICS,SBAOfficeLabel,Loan_Range,job_Range,full_add,Lat,Long,FIPS_z
0,0,514903,514903,514903,5731797102,4/14/20,951,PPP,"ASIA PACIFIC HOTELS, INC",PO Box 501029,...,999 Other,7211 Traveler Accommodation,7211.0,HAWAII DISTRICT OFFICE,d $2-5 million,100 to 249,"PO Box 501029, SAIPAN, MP",15.185048,145.746726,691100015001000
1,1,514904,514904,514904,4500217210,4/27/20,951,PPP,INTERPACIFIC RESORTS (SAIPAN) CORPORATION,502370 PO Box,...,999 Other,7211 Traveler Accommodation,7211.0,HAWAII DISTRICT OFFICE,c $1-2 million,250 to 499,"502370 PO Box, SAIPAN, MP",15.185048,145.746726,691100015001000
2,2,514905,514905,514905,5843458509,3/2/21,951,PPS,MICRONESIA RESORT INC.,CHALAN PALE ARNOLD SAN ROQUE,...,999 Other,7211 Traveler Accommodation,7211.0,HAWAII DISTRICT OFFICE,c $1-2 million,100 to 249,"CHALAN PALE ARNOLD SAN ROQUE, SAIPAN, MP",15.252495,145.779802,691100001002005
3,3,514906,514906,514906,4511357201,4/27/20,951,PPP,MICRONESIA RESORT INC.,5152 PO BOX,...,999 Other,7211 Traveler Accommodation,7211.0,HAWAII DISTRICT OFFICE,c $1-2 million,250 to 499,"5152 PO BOX, SAIPAN, MP",15.185048,145.746726,691100015001000
4,4,514907,514907,514907,8497628610,3/25/21,951,PPS,WORLD CORPORATION,World Resort Building Susupe Village Beach Road,...,999 Other,7211 Traveler Accommodation,7211.0,HAWAII DISTRICT OFFICE,c $1-2 million,250 to 499,World Resort Building Susupe Village Beach Roa...,15.158430,145.703139,691100008002003


In [26]:
##### fill in missing Lat, Long, FIPS
geo_MP_fips.loc[geo_MP_fips["FIPS_z"].isnull()] #27

,Unnamed: 0,Unnamed: 0.1,Unnamed: 0.1.1,Unnamed: 0.1.1.1,LoanNumber,DateApproved,SBAOfficeCode,ProcessingMethod,BorrowerName,BorrowerAddress,...,TRIIndustrySector,IndustrySubsector,4DigitNAICS,SBAOfficeLabel,Loan_Range,job_Range,full_add,Lat,Long,FIPS_z


In [27]:
geo_MP_fips["BusinessType"] = geo_MP_fips["BusinessType"].fillna('Not Available')
geo_MP_fips["BusinessType"].unique()
geo_MP_fips.to_csv("../../data/state_data/geo/geo_fips/MP_fips_scraped.csv") 
geo_MP_fips =pd.read_csv("../../data/state_data/geo/geo_fips/MP_fips_scraped.csv") 
geo_MP_fips["BusinessType"].unique()

array(['Corporation', 'Limited Liability Company(LLC)',
       'Non-Profit Organization', 'Subchapter S Corporation',
       'Sole Proprietorship'], dtype=object)